In [3]:
from convokit import Corpus, download
corpus = Corpus(filename=download("movie-corpus"))
import torch

In [4]:

print("总话语数:", len(corpus.utterances))
print("总对话数（会话数）:", len(corpus.conversations))


for conversation_id in corpus.conversations:
    conversation = corpus.get_conversation(conversation_id)
    print(f"对话ID: {conversation_id}")
    for utterance in conversation.iter_utterances():
        print(f"{utterance.speaker.id}: {utterance.text}")
    break  

总话语数: 304713
总对话数（会话数）: 83097
对话ID: L1044
u0: They do not!
u2: They do to!


In [5]:
conversations_texts = []

# 
for i, conversation_id in enumerate(corpus.conversations):
    if i >= 10000:  
        break
    conversation = corpus.get_conversation(conversation_id)
   
    conversation_text = ' '.join([utterance.text for utterance in conversation.iter_utterances()])
    conversations_texts.append(conversation_text)

In [6]:
import tiktoken
# create a tokenizer

encoding = tiktoken.get_encoding("cl100k_base")

In [11]:
tokenized_text = encoding.encode("".join(conversations_texts))

print(len(tokenized_text))

502498


In [23]:
#convert to tensor
tokenized_text = torch.tensor(tokenized_text)
print(tokenized_text.shape)

torch.Size([502498])


C:\Users\11632\AppData\Local\Temp\ipykernel_28280\1806850795.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_text = torch.tensor(tokenized_text)


In [24]:
#split the data into training and validation sets
train_idex = int(len(tokenized_text) * 0.9)
train_data = tokenized_text[:train_idex]
valid_data = tokenized_text[train_idex:]



In [25]:
batch_size = 8
context_size = 64
d_model = 64

In [26]:
#randomly extract a batch of data from train data
data = train_data
idxs = torch.randint(0 , len(data) - context_size, size = (batch_size,))
x_batch = torch.stack([data[idx:idx + context_size] for idx in idxs])
y_batch = torch.stack([data[idx + 1 :idx + context_size + 1] for idx in idxs])


In [27]:
import pandas as pd
pd.DataFrame(x_batch.numpy())

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,45446,358,2733,5128,16917,922,420,13,3011,10578,...,2684,596,264,2763,315,15525,17619,304,1070,11
1,1051,304,3021,13,220,1472,1440,1268,433,374,...,13,2876,422,279,1274,2349,3871,304,12976,311
2,4423,369,264,2697,1418,13,358,3194,499,13,...,264,5128,40666,5743,315,1667,13,2100,1131,499
3,9568,11,1095,757,1781,1131,3639,1051,499,5605,...,584,656,11,28146,30,578,892,374,4401,704
4,4333,30,2100,11,358,2322,264,502,4333,17182,...,430,499,3077,1027,43759,25492,315,701,24099,5253
5,1093,430,30,1102,1253,617,7077,11,719,539,...,10466,323,97387,279,4027,6798,704,315,279,10054
6,4438,596,430,1131,19418,430,387,264,1695,3245,...,539,36380,4954,757,13,358,11471,13,220,1226
7,220,16299,374,1633,6555,1606,433,706,279,2532,...,1131,9514,1440,1131,7566,0,1628,358,574,20910


In [29]:
encoding.decode(x_batch[1].numpy())

" were in love.  You know how it is when you laugh all the time. Then the relationship remains the same and the love changes only when there's change in the two people who share that love. Sure.  But that's only when the love itself goes unchanged. Not if the people change together in relation to"